<h1 align="center"><font color="yellow">LangChain: Agents 🤖</font></h1>

<font color="yellow">Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro</font>

Links de estudo:

* [LangChain Chat with Custom Tools, Functions and Memory](https://medium.com/@gil.fernandes/langchain-chat-with-custom-tools-functions-and-memory-e34daa331aa7)

* [How to connect LLM to SQL database with LangChain SQLChain](https://medium.com/dataherald/how-to-langchain-sqlchain-c7342dd41614)

* [Chat with SQL database via LangChain SQLDatabaseChain](https://github.com/sugarforever/LangChain-SQL-Chain/tree/main)

* [LangChain: pinecone-io](https://github.com/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/06-langchain-agents.ipynb)

<font color="orange">Os agentes são como "ferramentas" (`Tools`) para `LLMs`. Eles permitem que um LLM acesse a `Google Search`, execute cálculos complexos com `Python` e até mesmo faça consultas `SQL`.

Neste Notebook, exploraremos os `agentes` e como usá-los no `LangChain`.

Começaremos instalando as bibliotecas de pré-requisito que usaremos neste exemplo.</font>

```
!pip install -qU langchain openai google-search-results wikipedia sqlalchemy
```

In [1]:
%pip install -qU langchain openai google-search-results wikipedia sqlalchemy


Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip show langchain

Name: langchain
Version: 0.0.265
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /home/eddygiusepe/1_Eddy_Giusepe/3_estudando_LLMs/Large_Language_Models_LLMs/venv_LLMs/lib/python3.10/site-packages
Requires: aiohttp, async-timeout, dataclasses-json, langsmith, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import openai
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key  = os.getenv('OPENAI_API_KEY')


In [4]:
from langchain import OpenAI


Eddy_API_KEY_OpenAI = os.environ['OPENAI_API_KEY'] 

llm = OpenAI(
    openai_api_key=Eddy_API_KEY_OpenAI,
    temperature=0
)

<font color="orange">Como fizemos antes, contaremos nossos tokens em cada chamada.</font>

In [5]:
from langchain.callbacks import get_openai_callback

def count_tokens(agent, query):
    with get_openai_callback() as cb:
        result = agent(query)
        print(f'Gastou um total de {cb.total_tokens} Tokens')

    return result

<font color="orange">Com tudo isso configurado, vamos pular para Agentes.</font>


<font color="red">O que é um agente?</font>

<font color="pink">Definição:</font> 

A chave por trás dos `agentes` é dar aos LLM's a possibilidade de usar ferramentas em seu fluxo de trabalho. É aqui que o langchain se afasta da popular implementação do `ChatGPT` e podemos começar a ter uma ideia do que ele nos oferece como construtores. Até agora, cobrimos vários blocos de construção isoladamente. Vamos vê-los ganhar vida.

A definição oficial de agentes é a seguinte:

>> Os agentes usam um LLM para determinar quais ações devem ser executadas e em que ordem. Uma ação pode ser usar uma ferramenta e observar sua saída ou retornar ao usuário.

Nesta edição, abordaremos o que podemos chamar de agentes "genéricos" que realmente são capazes de realizar muitas metatarefas. Existem outros agentes mais específicos que são ajustados para tarefas diferentes (chamados `kits de ferramentas de agente`), mas iremos abordá-los em uma edição futura.


# <font color="red">Criar um Database</font>

<font color="orange">Usaremos os agentes para interagir com um pequeno banco de dados de amostras de ações. Não vamos nos aprofundar nos detalhes porque esta é apenas uma ferramenta fictícia que construiremos para fins ilustrativos. Vamos criá-lo.</font>

In [6]:
from sqlalchemy import MetaData

metadata_obj = MetaData()


from sqlalchemy import Column, Integer, String, Table, Date, Float

stocks = Table(
    "stocks",
    metadata_obj,
    Column("obs_id", Integer, primary_key=True),
    Column("stock_ticker", String(4), nullable=False),
    Column("price", Float, nullable=False),
    Column("date", Date, nullable=False),    
)



from sqlalchemy import create_engine

engine = create_engine("sqlite:///database_eddy.db")
metadata_obj.create_all(engine)



In [7]:
from datetime import datetime

observations = [
    [1, 'ABC', 200, datetime(2023, 1, 1)],
    [2, 'ABC', 208, datetime(2023, 1, 2)],
    [3, 'ABC', 232, datetime(2023, 1, 3)],
    [4, 'ABC', 225, datetime(2023, 1, 4)],
    [5, 'ABC', 226, datetime(2023, 1, 5)],
    [6, 'XYZ', 810, datetime(2023, 1, 1)],
    [7, 'XYZ', 803, datetime(2023, 1, 2)],
    [8, 'XYZ', 798, datetime(2023, 1, 3)],
    [9, 'XYZ', 795, datetime(2023, 1, 4)],
    [10, 'XYZ', 791, datetime(2023, 1, 5)],
]

In [8]:
from sqlalchemy import insert

def insert_obs(obs):
    stmt = insert(stocks).values(
    obs_id=obs[0], 
    stock_ticker=obs[1], 
    price=obs[2],
    date=obs[3]
    )

    with engine.begin() as conn:
        conn.execute(stmt)
        

In [ ]:
for obs in observations:
    insert_obs(obs)
    

In [41]:
# https://python.langchain.com/docs/use_cases/sql
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain


db = SQLDatabase(engine)
sql_chain = SQLDatabaseChain.from_llm(llm=llm, db=db, verbose=True)



<font color="orange">Por fim, criaremos uma ferramenta com essa cadeia (`chain`). Para criar uma ferramenta personalizada, precisamos apenas de três entradas:

* `name:` o nome da ferramenta (enviado para o llm como contexto)

* `func:` a função que será aplicada na requisição do `LLM`

* `description:` a descrição da ferramenta, para que ela deve ser usada (enviada para o llm como contexto)</font>

In [42]:
from langchain.agents import Tool

sql_tool = Tool(
    name='Stock DB',
    func=sql_chain.run,
    description="Útil para quando você precisa responder a perguntas sobre ações e seus preços."
)


# <font color="red">Tipos de Agente</font>

Nesta seção, revisaremos vários agentes e veremos como eles 'pensam' e o que podem fazer.

O uso de um dos agentes pré-construídos do `langchain`` envolve três variáveis:

* Definindo as ferramentas (`Tools`)

* Definindo o llm

* Definindo o tipo de agente

Isso tudo é muito fácil de fazer no `langchain``, como veremos no exemplo a seguir.



## <font color="yellow">1. Tipo de Agente `Zero Shot React`</font>

Usaremos apenas algumas ferramentas, mas você pode combinar as que preferir. Lembre-se de que as ferramentas são apenas correntes (`chains`) de utilidade sob o capô, que são correntes que servem a um propósito específico.

Primeiro precisamos inicializar as ferramentas (`tools`) que usaremos neste exemplo. Aqui queremos que nosso agente faça `matemática`, então usaremos `'llm-math'`, a ferramenta que temos para resolver problemas matemáticos. Esta é uma das várias ferramentas que podemos carregar com o utilitário `load_tools`.

In [47]:
from langchain.agents import load_tools

tools = load_tools(
    ["llm-math"], 
    llm=llm
)

<font color="orange">Para poder pesquisar os preços de nossas ações, também usaremos a ferramenta personalizada que construímos anteriormente com os dados `sql` ('Stock DB'). Acrescentaremos esta Tool à nossa lista de Tools.</font>

In [48]:
tools.append(sql_tool)

Como o nome sugere, usaremos este agente para executar tarefas de `'zero shot'` na entrada. Isso significa que não teremos várias interações interdependentes, mas apenas uma. `Em outras palavras, este agente não terá memória`.

Agora estamos prontos para inicializar o agente! Usaremos `verbose` em `True` para que possamos ver qual é o processo de 'pensamento' do nosso agente.


<font color="red">Observação Importante:</font> 

Ao interagir com os agentes, é muito importante definir os parâmetros `max_iterations` porque os agentes podem ficar presos em loops infinitos que consomem muitos tokens. O valor padrão é `15` para permitir muitas Tools e raciocínio complexo, mas para a maioria dos aplicativos você deve mantê-lo bem mais baixo.


In [51]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description", 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=3,
)


<font color="orange">Vamos ver nosso agente recém-criado em ação! Faremos uma pergunta que envolve uma operação matemática sobre os preços das ações.</font>

In [52]:
result = count_tokens(
    zero_shot_agent, 
    "Qual é a multiplicação da relação entre ações de " +
    "preços de 'ABC' e 'XYZ' em 3 de janeiro e a relação " + 
    "entre os mesmos preços de ações (stock) em 4 de janeiro?"
)




> Entering new AgentExecutor chain...
 I need to find the ratio between the two stocks on 3 January and the ratio between the same stocks on 4 January.
Action: Stock DB
Action Input: Prices of 'ABC' and 'XYZ' on 3 January and 4 January

> Entering new SQLDatabaseChain chain...
Prices of 'ABC' and 'XYZ' on 3 January and 4 January
SQLQuery:SELECT stock_ticker, price, date FROM stocks WHERE stock_ticker IN ('ABC', 'XYZ') AND date IN ('2023-01-03', '2023-01-04');
SQLResult: [('ABC', 232.0, '2023-01-03'), ('ABC', 225.0, '2023-01-04'), ('XYZ', 798.0, '2023-01-03'), ('XYZ', 795.0, '2023-01-04')]
Answer:On 3 January, ABC was 232.0 and XYZ was 798.0. On 4 January, ABC was 225.0 and XYZ was 795.0.
> Finished chain.

Observation: On 3 January, ABC was 232.0 and XYZ was 798.0. On 4 January, ABC was 225.0 and XYZ was 795.0.
Thought: I need to calculate the ratio between the two stocks on 3 January and the ratio between the same stocks on 4 January.
Action: Calculator
Action Input: 232.0/798.0 and

In [ ]:
print(zero_shot_agent.agent.llm_chain.prompt.template)

A pergunta que devemos nos fazer aqui é: `como os agentes são diferentes das cadeias?`

Se observarmos a lógica do agente e o prompt que acabamos de imprimir, veremos algumas diferenças claras. `Primeiro`, temos as Tools incluídas no prompt. Em `segundo` lugar, temos um processo de pensamento que antes era imediato em chains, mas agora envolve uma sequência de `'pensamento'`, `'ação'`, `'entrada de ação'`, `'observação'`. <font color="red">O que é isso tudo?</font>


Por enquanto, basta dizer que o `LLM` agora tem a capacidade de 'raciocinar' sobre a melhor forma de usar as Tools para resolver nossa consulta e pode combiná-las de maneira inteligente com apenas uma breve descrição de cada uma delas. Se você quiser saber mais sobre esse paradigma (`MRKL`) em detalhes, consulte este [paper](https://arxiv.org/pdf/2205.00445.pdf).



Finalmente, vamos prestar atenção ao `'agent_scratchpad'`. <font color="red">O que é aquilo?</font> Bem, é onde estaremos anexando cada pensamento ou ação que o agente já realizou. Desta forma, a cada momento, o agente saberá o que descobriu e poderá continuar seu processo de pensamento. Em outras palavras, depois de usar uma ferramenta, ela adiciona seus pensamentos e observações ao bloco de rascunho e continua a partir daí.



## <font color="yellow">2. Tipo de Agente `Conversational React`</font>

<font color="orange">O `agente zero shot` é bem interessante mas, como dissemos antes, `não tem memória`. E se quisermos um assistente que se lembre de coisas sobre as quais conversamos e também possa raciocinar sobre elas e usar ferramentas? Para isso temos o `agente conversational react`.

Usaremos as mesmas ferramentas que especificamos anteriormente:</font>

In [54]:
tools = load_tools(
    ["llm-math"], 
    llm=llm
)


tools.append(sql_tool)


<font color="orange">O tipo de memória que está sendo usado aqui é um `buffer de memória simples` para nos permitir lembrar as etapas anteriores na cadeia de raciocínio. Para mais informações sobre memória veja a documentação do LangChain.</font>

In [61]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")


conversational_agent = initialize_agent(
    agent='conversational-react-description', 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory,
)


In [62]:
result = count_tokens(
    conversational_agent, 
    "Forneça-me os preços das ações da ABC em 1º de janeiro"
                     )



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Stock DB
Action Input: ABC stock prices on January 1st

> Entering new SQLDatabaseChain chain...
ABC stock prices on January 1st
SQLQuery:SELECT price FROM stocks WHERE stock_ticker = 'ABC' AND date = '2023-01-01' LIMIT 5;
SQLResult: [(200.0,)]
Answer:The ABC stock price on January 1st was 200.0.
> Finished chain.

Observation: The ABC stock price on January 1st was 200.0.
Thought: Do I need to use a tool? No
AI: The ABC stock price on January 1st was 200.0.

> Finished chain.
Gastou um total de 1904 Tokens


In [63]:
print(conversational_agent.agent.llm_chain.prompt.template)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks 

<font color="orange">O assistente de conversação é voltado para a conversa e tem um pouco mais de dificuldade em combinar uma combinação complexa de Tools. Vamos ver o que acontece se tentarmos responder à mesma pergunta que fizemos ao nosso agente anterior:</font>

In [ ]:
result = count_tokens(
    conversational_agent, 
    "Qual é a multiplicação da relação entre ações de preços de 'ABC' e 'XYZ' em 3 de janeiro e a relação entre os mesmos preços de ações (stock) em 4 de janeiro?"
)


Como podemos ver, ele resolve o problema, mas de uma maneira diferente, usando a ferramenta SQL para resolver também a matemática. Interessante!

## <font color="yellow">3. Tipo de Agente `React Docstore`</font>

Este tipo de agente é semelhante aos que vimos até agora, mas inclui a interação com um `docstore`. Terá à sua disposição duas e apenas duas Tools: `'Search'` e `'Lookup'`.

Com `'Search'` ele trará um artigo relevante e com `'Lookup'` o agente encontrará a informação certa no artigo. Isso provavelmente é mais fácil de ver em um exemplo:

In [67]:
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer

docstore=DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description='search wikipedia'
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description='lookup a term in wikipedia'
    )
]


In [68]:
docstore_agent = initialize_agent(
    tools, 
    llm, 
    agent="react-docstore", 
    verbose=True,
    max_iterations=3
)


In [71]:
count_tokens(docstore_agent, "Qual é a famosa frase do Arquimedes?")



> Entering new AgentExecutor chain...
Thought: Eu preciso pesquisar Arquimedes e encontrar sua famosa frase.
Action: Search[Arquimedes]
Observation: Archimedes of Syracuse (, ARK-ihm-EE-deez; c. 287 – c. 212 BC) was an Ancient Greek mathematician, physicist, engineer, astronomer, and inventor from the ancient city of Syracuse in Sicily. Although few details of his life are known, he is regarded as one of the leading scientists in classical antiquity. Considered the greatest mathematician of ancient history, and one of the greatest of all time, Archimedes anticipated modern calculus and analysis by applying the concept of the infinitely small and the method of exhaustion to derive and rigorously prove a range of geometrical theorems. These include the area of a circle, the surface area and volume of a sphere, the area of an ellipse, the area under a parabola, the volume of a segment of a paraboloid of revolution, the volume of a segment of a hyperboloid of revolution, and the area of 

{'input': 'Qual é a famosa frase do Arquimedes?',
 'output': '"Eureka! Eu encontrei-o!"'}

<font color="orange">Não imprimiremos o prompt aqui porque é muito grande, mas você mesmo pode ver se quiser (já sabemos como fazer).</font>

Resumindo, contém vários exemplos do loop `Question` > `Thought`> `Action` > `Observation`, que incluem as ferramentas `Search` e `Lookup`.

Se você quiser aprender mais sobre essa abordagem, este é o [paper](https://arxiv.org/pdf/2210.03629.pdf) do `ReAct`.

## <font color="yellow">4. Tipo de Agente `Self Ask with Search`</font>

Este é o agente de primeira escolha a ser usado ao usar `LLMs` para extrair informações com um mecanismo de pesquisa (search engine). O agente fará perguntas de acompanhamento e usará a funcionalidade de pesquisa para obter respostas intermediárias que o ajudarão a chegar a uma resposta final.

In [72]:
from langchain import OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, Tool


search = SerpAPIWrapper(serpapi_api_key='api_key')
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description='google search'
    )
]

self_ask_with_search = initialize_agent(tools, llm, agent="self-ask-with-search", verbose=True)

<font color="orange">Não iremos interagir com este agente pois para isso precisaríamos de uma `chave serpapi`. No entanto, verificando o prompt, podemos ver alguns exemplos de como ele funciona:</font>

In [73]:
print(self_ask_with_search.agent.llm_chain.prompt.template)

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin